# Diffusion of a Gaussian Function, Failure Due to Bad Boundary Conditions

This notebook solves the diffusion of a Gaussian hill with inappropriate boundary conditions, leading to poor finite element analysis (FEA) results. The initial condition is:
\begin{align}
    u_0(x,y) = e^{-5x^2 - 5y^2}
\end{align}
on the domain $[-2,2]\times[-2,2]$. Instead of homogeneous Dirichlet boundary conditions ($u_D = 0$), we use Neumann boundary conditions ($\nabla u \cdot n = 0$) on most boundaries, with a single-point Dirichlet condition ($u = 0$ at $(-2, -2)$) to avoid a singular system. This traps the solution, preventing proper diffusion and causing unphysical results.

The mesh resolution is set to $nx, ny = 20, 20$ to ensure the mesh is not the primary source of error, isolating the effect of bad boundary conditions. Visualizations (PyVista GIF and XDMF for Paraview) show the unphysical solution.

In [ ]:
import matplotlib as mpl
import pyvista
import ufl
import numpy as np
from petsc4py import PETSc
from mpi4py import MPI
from dolfinx import fem, mesh, io, plot
from dolfinx.fem.petsc import assemble_vector, assemble_matrix, create_vector, apply_lifting, set_bc

# Define temporal parameters
t = 0
T = 1.0
num_steps = 50
dt = T / num_steps

# Define mesh (reasonable resolution to isolate boundary condition effects)
nx, ny = 20, 20  # Increased from 2,2 to ensure mesh is not the primary issue
domain = mesh.create_rectangle(MPI.COMM_WORLD, [np.array([-2, -2]), np.array([2, 2])], [nx, ny], mesh.CellType.triangle)
V = fem.functionspace(domain, ("Lagrange", 1))

## Initial and Boundary Conditions

The initial condition is a Gaussian hill. The boundary conditions are deliberately incorrect: Neumann ($\nabla u \cdot n = 0$) on most boundaries (implicitly applied by omitting Dirichlet BCs), with a single-point Dirichlet condition at $(-2, -2)$ to ensure solver convergence. This prevents the Gaussian from diffusing properly, leading to unphysical results.

In [ ]:
# Create initial condition
def initial_condition(x, a=5):
    return np.exp(-a * (x[0]**2 + x[1]**2))

u_n = fem.Function(V)
u_n.name = "u_n"
u_n.interpolate(initial_condition)

# Create boundary condition (bad: Neumann on most boundaries, minimal Dirichlet)
fdim = domain.topology.dim - 1
# Single-point Dirichlet BC at (-2, -2) to avoid singular system
def fixed_point(x):
    return np.logical_and(np.isclose(x[0], -2), np.isclose(x[1], -2))
u_zero = PETSc.ScalarType(0)
bc = fem.dirichletbc(u_zero, fem.locate_dofs_geometrical(V, fixed_point), V)

## Time-Dependent Output

We use an `XDMFFile` to store the solution for Paraview visualization and PyVista for real-time GIF animation. The XDMF file stores the mesh once and appends solutions, reducing storage. The GIF will show the unphysical diffusion caused by bad boundary conditions.

In [ ]:
xdmf = io.XDMFFile(domain.comm, "bad_boundary_diffusion.xdmf", "w")
xdmf.write_mesh(domain)

# Define solution variable and interpolate initial solution
uh = fem.Function(V)
uh.name = "uh"
uh.interpolate(initial_condition)
xdmf.write_function(uh, t)

## Variational Problem and Solver

The variational problem is the same as the original diffusion problem, but the Neumann boundary conditions (implicitly applied) cause incorrect behavior. We use a PETSc solver with LU factorization for robustness.

In [ ]:
u, v = ufl.TrialFunction(V), ufl.TestFunction(V)
f = fem.Constant(domain, PETSc.ScalarType(0))
a = u * v * ufl.dx + dt * ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = (u_n + dt * f) * v * ufl.dx

# Assemble forms
bilinear_form = fem.form(a)
linear_form = fem.form(L)

# Assemble matrix and vector
A = assemble_matrix(bilinear_form, bcs=[bc])
A.assemble()
b = create_vector(linear_form)

# Create solver
solver = PETSc.KSP().create(domain.comm)
solver.setOperators(A)
solver.setType(PETSc.KSP.Type.PREONLY)
solver.getPC().setType(PETSc.PC.Type.LU)

## Visualization with PyVista

We create a GIF animation (`bad_boundary_u_time.gif`) to visualize the solution’s evolution. The warped mesh will highlight the unphysical behavior (e.g., Gaussian hill not diffusing properly, boundary artifacts). The solution will persist longer than expected due to Neumann BCs trapping the mass.

In [ ]:
pyvista.start_xvfb()
grid = pyvista.UnstructuredGrid(*plot.vtk_mesh(V))
plotter = pyvista.Plotter()
plotter.open_gif("bad_boundary_u_time.gif", fps=10)

grid.point_data["uh"] = uh.x.array
warped = grid.warp_by_scalar("uh", factor=1)
viridis = mpl.colormaps.get_cmap("viridis").resampled(25)
sargs = dict(title_font_size=25, label_font_size=20, fmt="%.2e", color="black",
             position_x=0.1, position_y=0.8, width=0.8, height=0.1)
plotter.add_mesh(warped, show_edges=True, lighting=False, cmap=viridis, scalar_bar_args=sargs,
                 clim=[0, max(uh.x.array)])
plotter.write_frame()

## Time-Stepping and Output

The time-stepping loop solves the problem, updates the solution, and saves results to the XDMF file and GIF. We print the maximum solution value to quantify the bad results (expect slow decay compared to correct Dirichlet BCs).

In [ ]:
for i in range(num_steps):
    t += dt

    # Update right-hand side
    with b.localForm() as loc_b:
        loc_b.set(0)
    assemble_vector(b, linear_form)
    apply_lifting(b, [bilinear_form], [[bc]])
    b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    set_bc(b, [bc])

    # Solve
    solver.solve(b, uh.x.petsc_vec)
    uh.x.scatter_forward()

    # Update previous solution
    u_n.x.array[:] = uh.x.array

    # Write to file
    xdmf.write_function(uh, t)

    # Update plot
    new_warped = grid.warp_by_scalar("uh", factor=1)
    warped.points[:, :] = new_warped.points
    warped.point_data["uh"][:] = uh.x.array
    plotter.write_frame()

    # Print max solution value to check bad results
    print(f"Time {t:.2f}, Max u: {np.max(uh.x.array):.2e}")

plotter.close()
xdmf.close()

![gif](poor_boundary.gif)
## Expected Results and Verification

The solver will converge, but the solution will be unphysical due to Neumann boundary conditions trapping the Gaussian hill. Expected issues:
- **Persistent High Values**: The max solution value will decay slowly (e.g., max $u \approx 0.5$–$0.8$ at $t=1.0$, vs. $<0.1$ with correct Dirichlet BCs).
- **Boundary Artifacts**: Oscillations or high values near boundaries in the GIF and Paraview output.
- **Visualization**: The GIF (`bad_boundary_u_time.gif`) and XDMF file (`bad_boundary_diffusion.xdmf`) will show a Gaussian that doesn’t flatten properly, with trapped mass.

To verify in Paraview:
1. Open `bad_boundary_diffusion.xdmf` (`File -> Open`, `Apply`).
2. Add `Annotate Time` (`Sources -> Alphabetical -> Annotate Time`, `Apply`).
3. Save animation (`File -> Save Animation`, AVI, 5–10 fps).
4. Look for persistent high values and boundary artifacts.